### Fase 1: La Plaza 
Te encuentras en la plaza de la ciudad, donde los habitantes desarrollan sus diversas profesiones: panaderos, herreros, guardias, escuderos...   
En el siguiente enlace https://cutt.ly/peVAXig  encontrarás un archivo que contiene un número por cada persona que se encuentra en la plaza. Suma los números que correspondan a los guardias y podrás pasar a la siguiente fase.


In [1]:
import pandas as pd

In [4]:
plaza = pd.read_csv('people.csv',delimiter=";")
plaza

,Nombre,Prof,Num
0,Luca,Caballero,5
1,Aalis,Guardia,1
2,Aethelu,Guardia,9
3,Agnes,Campesino,2
4,Amice,Labrador,4
5,Dionisio,Pastor,7
6,Addy,Guardia,3
7,Aloys,Cerrajero,4
8,Amis,Herrero,5
9,Blazh,Campesino,6


In [5]:
plaza.loc[plaza.Prof=="Guardia","Num"].sum()

44

### Fase 2: La Biblioteca 
Te encuentras en la biblioteca. La cantidad de libros es inmensa. Por suerte, en la plaza conseguiste lo que necesitas. Encuentra el libro que esté en la posición del número que has guardado y accede a su contenido.   
Descarga el archivo books.xml creando una conexión al servidor FTP  
`Servidor: f31-preview.runhosting.com  
Usuario: 4009006_DATOS   
Contraseña: Rafa9999 `

In [6]:
import ftplib

In [7]:
ftp = ftplib.FTP('f31-preview.runhosting.com','4009006_DATOS','Rafa9999')

In [8]:
ftp.retrbinary("RETR "+"books.xml",open('books.xml','wb').write)

'226 Transfer complete'

In [9]:
ftp.quit()

'221 Goodbye.'

In [10]:
import xml.etree.ElementTree as ET
tree = ET.parse('books.xml')

In [16]:
lst = tree.findall('book')
print(len(lst))
print(lst[43].find('link').text)

100
http://johnstoniatexts.x10host.com/homer/iliad1html.html


### Fase 3: El Libro 
¡Rápido! El tiempo se agota. Para pasar a la siguiente frase deberás encontrar la palabra de seis letras que más se repite en el libro. ¡Usa tu magia!

In [30]:
import urllib.request
from bs4 import BeautifulSoup
url = 'http://johnstoniatexts.x10host.com/homer/iliad1html.html'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html)
tags = soup.find_all('span',class_="auto-style21")

texto = ''
for tag in tags:
    try: texto += tag.contents[0]
    except: None

texto=texto.replace(',','').replace('.','').replace(':','').replace('?','').replace('(','').replace(')','').replace('','').replace('\xa0','') # Elimino signos de (puntos, comas, etc.)
palabras = texto.lower().split()

contadores = dict()
for palabra in palabras:
    if len(palabra)==6:
        contadores[palabra] = contadores.get(palabra,0) + 1
        
df = pd.DataFrame(list(contadores.items()),columns=['palabra','contador']).sort_values('contador',ascending=False)
df

,palabra,contador
1,apollo,13
6,sacred,9
37,before,8
74,should,8
53,that’s,7
...,...,...
135,killer,1
137,finest,1
138,man”so,1
140,facing,1


### Fase 4: La Tienda   
Tu siguiente destino es la tienda de comestibles cercana a la Iglesia. Solo te dejarán entrar si conoces la contraseña, la cual conseguiste en la fase anterior. Una vez dentro, averigua cuál es el producto más vendido en la historia de la tienda, y cómpralo. 
Descarga el archivo del siguiente enlace: https://cutt.ly/peVALXr, descomprímelo utilizando la respuesta de la fase anterior como contraseña. Carga la base de datos en Python, y encuentra el nombre 
del producto más vendido.


In [31]:
import sqlite3
import pandas as pd

In [32]:
conn = sqlite3.connect('tienda.sqlite')

In [33]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',conn)

,name
0,Employee
1,Category
2,Customer
3,Shipper
4,Supplier
5,Order
6,Product
7,OrderDetail
8,CustomerCustomerDemo
9,CustomerDemographic


In [46]:
query='''
SELECT ProductName,ProductId,SUM(Quantity) AS Total
FROM OrderDetail DET
JOIN Product PRO
on DET.ProductId = PRO.Id
GROUP BY ProductId
ORDER BY Total DESC
LIMIT 1
'''
pd.read_sql(query,conn)

,ProductName,ProductId,Total
0,Camembert Pierrot,60,1577


### FASE 5: El Castillo   
¡Ya casi estás! Solo te falta entrar al castillo. Para ello, deberás sobornar al guardia elaborando la receta más calórica con el alimento que conseguiste en la fase anterior.

In [48]:
import urllib.request
import json
api_id = '0ceaaf83'
api_key = '93ff26f92c228dbafa3d7d2f9b789a39'

url = 'https://api.edamam.com/search'
q = 'Camembert'

url2 = url+'?q='+q+'&app_id='+api_id+'&app_key='+api_key
print(url2)

datos = urllib.request.urlopen(url2).read().decode()
js = json.loads(datos)


https://api.edamam.com/search?q=Camembert&app_id=0ceaaf83&app_key=93ff26f92c228dbafa3d7d2f9b789a39


In [51]:
recetas =[]
calorias = []
for receta in js['hits']:
    recetas.append(receta['recipe']['label'])
    calorias.append(receta['recipe']['calories'])

In [53]:
pd.DataFrame({'recetas':recetas,
              'calorias':calorias}).sort_values('calorias',ascending=False)

,recetas,calorias
4,Baked camembert,5911.433894
2,Puff pastry tarts with broccoli and Camembert ...,3147.970693
6,Baked Camembert with Thyme & Garlic,1672.991124
8,Ham and Camembert Baguette Sandwich recipes,1416.203587
0,Baked Camembert with Fresh Fruit,1250.515222
3,Camembert Fondue Recipe,1027.489117
5,Baked Camembert with garlic bread,914.115000
9,Camembert Baked In Its Box,794.432840
7,Candied Camembert,632.250000
1,Pan-fried camembert sandwich,568.392857
